Simplest form of Agentic RAG - the router.

Given a query router will pick one of serval query entrants to execute a query.

Router Engine

What is summary of MetaGPT document?

will build router which will handle both QandA and Summerization

Router 

1. Q&A - > Query Engine - > Vector index < - MetaGPT document

2. Summerization - > Summary index < - MetaGPT Document


In [1]:
import os
from dotenv import load_dotenv


In [2]:
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 8


In [3]:
import nest_asyncio # jupiter runs an event loop behind the scenes,and lost of modules use async and to make async play nice we need to import this.
nest_asyncio.apply()

In [6]:
#sample document we will use metaGPT 2024 paper on multiagent framework
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["/Users/apple/Desktop/project/LLM/agentic_RAG/Data/metagpt.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size = 1024)  # split in order of sentences
nodes = splitter.get_nodes_from_documents(documents)

In [8]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

build index :- index is metadata over our data.

we will define 2 indexes summary index and vector index.

when you query an index, different indexes will have different retrevial behaviors.

Vector Index :- indexes nodes via text embeddings and its core abstraction and Llamaindex, and core abstraction for building any sort of RAG system.

Querying the vector index will return the most similar nodes by embedding similarity.

Summary index :- Querying summary index will return all the nodes current in the index, so it doesn't necessary depend on the user query,

but will return all the nodes that's currently in the index.


In [13]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

convert these indexes into query engines

Each query engines represents, overall query interface over the data that's store in this index, and combines retrieval with LLM synthesis

Each query engine is good for certain type of question, great use case for router, which can route dynamically between these different query entrants.


In [14]:
summary_query_engine = summary_index.as_query_engine(
    response_mode = "tree_summarize",
    use_async = True,
)

vector_query_engine = vector_index.as_query_engine()

convert to query tools:- 

A query tool now is just the query engine with metadata,specifically a description of what type of questions the tool can answer.

In [15]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine = summary_query_engine,
    description = (
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine = vector_query_engine,
    description = (
        "useful for retrieving specific context from the MetaGPT paper."
    ),
)

Selectors

There are different types of selectors to enable you to build a router.

Each of these selectors will have distinct attributes.

1. LLm selector, involves prompting an LLM to output a json that is then parsed and the corresponding indexes are queried.

2, Pydantic selector, use the openai function calling API to produce pydantic selection objects, rather than parsing raw JSON.

In [16]:
#LLM single selector

from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector = LLMSingleSelector.from_defaults(),
    query_engine_tools = [
        summary_tool,
        vector_tool,
    ],
    verbose = True
)

In [17]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The document is asking for a summary, which is typically related to summarization questions..
The document discusses the development of MetaGPT, a meta-programming framework that utilizes Standardized Operating Procedures (SOPs) to enhance multi-agent systems based on Large Language Models (LLMs). It introduces key roles in the software development process such as Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The framework incorporates efficient workflows, structured communication interfaces, and an executable feedback mechanism to enhance code generation quality. Additionally, it details the creation of a Python GUI application called a color meter, outlining its requirements, design, implementation approach, and testing strategy. The color meter allows users to select colors on the screen and view their RGB values in real-time, featuring a simple and user-friendly interface. The document also discusses the use of libraries like Tkin

In [19]:
print(len(response.source_nodes)) # here we can see length is same of number of chunks in entire documents

34


In [20]:
response = query_engine.query(
    "How do agents share information with other agents?"
)

print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context from the MetaGPT paper, which would likely include information on how agents share information..
Agents share information with other agents by utilizing a shared message pool where they publish structured messages. This shared message pool allows all agents to exchange messages directly, enabling them to both publish their own messages and access messages from other agents transparently. This method eliminates the need for agents to inquire about other agents and wait for their responses, thus enhancing communication efficiency.
